In [ ]:
import numpy as np
import pandas as pd
from gtfparse import read_gtf
import pysam as ps
from Bio.Seq import Seq

In [ ]:
prefix_variant_file = 'transcripts_under_10_percent_expression'

In [ ]:
# convert plasmid positions into genomic positions

# plasmid to genomic position table
plasmid_gen_table = pd.read_excel('00_input_data/genom_plasmid_pos_table.xlsx')

# variant table
in_plasmid_pos = pd.read_excel('00_input_data/'+prefix_variant_file+'_in_plasmid_pos.xlsx')

# define output file name
out_pos_file = '02_output_analysis/00_converted_pos/'+prefix_variant_file+'_for_cDNA.xlsx'

def convert_position(test_position, plasmid_genomic_table = plasmid_gen_table):
    converted_pos = 0
    
    for strand, start_g, end_g, start_p, end_p in zip(
        plasmid_genomic_table['strand'], plasmid_genomic_table['start_gen'], plasmid_genomic_table['end_gen'],
        plasmid_genomic_table['start_plasmid'], plasmid_genomic_table['end_plasmid']):
        
        if test_position >= start_p and test_position <= end_p:
            # distance to start position of exon/intron
            dist_start = test_position - start_p
            if strand == 'minus':
                # converted position is end position of genomic exon/intron minus distance
                converted_pos = end_g - dist_start
            elif strand == 'plus':
                # converted position is start position of genomic exon/intron plus distance
                converted_pos = start_g + dist_start
            else:
                print('Check if strand column is filled correctly (minus or plus).')
        
    return converted_pos


def output_position_file(out_positions_file, in_plasmid_positions = in_plasmid_pos):
    # SDHB specific (if strand == 'plus': exchange pos_last and pos_next)
    in_plasmid_positions['pos_next'] = in_plasmid_positions['plasmid_pos_last'].apply(convert_position)
    in_plasmid_positions['pos_last'] = in_plasmid_positions['plasmid_pos_next'].apply(convert_position)

    in_plasmid_positions['strand'] = 'minus'
    in_plasmid_positions['HUGO_Symbol'] = 'SDHB'
    in_plasmid_positions['kind'] = in_plasmid_positions['kind'].str.lower()

    in_plasmid_positions[['#CHROM', 'POS', 'REF', 'ALT', 'pos_last', 'pos_next', 'kind', 'shortcut', 
                          'strand', 'HUGO_Symbol']].to_excel(out_positions_file, index=False)

output_position_file(out_pos_file)

In [ ]:
variant_table_name = out_pos_file

In [ ]:
# whole reference sequence
ref_seq = ps.FastaFile('Hsapiens/GRCh38/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa')

# table with variants and positions
variants = pd.read_excel(variant_table_name)
# which reference transcript
variants['RefSeq accession'] = 'NM_003000'

# GTF file with sequence information
# gtf_df = read_gtf('Hsapiens/GRCh38/ensembl/GRCh38_latest_genomic.gtf')
# only those reference transcripts within variant file
# sdhb_gtf = gtf_df[(gtf_df['transcript_id'].str.startswith('NM_003000'))&(gtf_df['feature']=='CDS')]
# sdhb_gtf['transcript_id'] = sdhb_gtf['transcript_id'].str.split('.', expand=True)[0]
# sdhb_gtf['seqname'] = sdhb_gtf['seqname'].str.split('1.1', expand=True)[1]
# sdhb_gtf.to_pickle('sdhb_gtf_pickle')

# sdhb_gtf = pd.read_pickle('01_dataframes/sdhb_gtf_pickle').reset_index(drop=True)
# sdhb_gtf['diff_end_start'] = sdhb_gtf['end'] - sdhb_gtf['start'] + 1
# sdhb_gtf['exon_number'] = sdhb_gtf['exon_number'].astype(int)

# list_comb_cod_end = []
# for num in sdhb_gtf['exon_number']:
#     in_diff_sum = 0
#     for j in range(num):
#         in_diff_sum += sdhb_gtf[sdhb_gtf['exon_number']==j+1].iloc[0]['diff_end_start']
#     list_comb_cod_end.append(in_diff_sum)
    
# sdhb_gtf['end_coding_num'] = list_comb_cod_end
# sdhb_gtf['start_coding_num'] = sdhb_gtf['end_coding_num'] - sdhb_gtf['diff_end_start'] + 1
# sdhb_gtf[['feature','start','end','exon_number', 'diff_end_start', 'end_coding_num', 'start_coding_num']]

# sdhb_gtf.to_pickle('01_dataframes/sdhb_gtf_pickle')

# GTF file with sequence information for SDHB
sdhb_gtf = pd.read_pickle('01_dataframes/sdhb_gtf_pickle')

In [ ]:
def search_col(df, inp):
    return [i for i in df.columns if inp in i.lower()]

def rev_comp(seq):
    return str(Seq(seq).reverse_complement())

In [ ]:
# new cDNA/protein sequence with indels and variants
def cDNA(ind):
    cdna = ''
    ind_df = sdhb_gtf
    kind = variants.iloc[ind]['kind']
    chrom = str(variants.iloc[ind]['#CHROM'])
    var_pos = int(variants.iloc[ind]['POS'])
    ref_len = len(variants.iloc[ind]['REF'])-1
    pos_last = int(variants.iloc[ind]['pos_last'])
    pos_next = int(variants.iloc[ind]['pos_next'])
    ori = list(ind_df['strand'])[0]
    aa_pos = 0
    hgsv_pos = 0
    
    # exclude duplicated sequences
    new_pos = 0
    
    # for new HGVSg in VCF
    ref_n = ''
    alt_n = ''
    
    # for coding positions, differences
    cod_start_n = ''
    cod_end_n = ''
    cod_diff = 0
    
    cdna_hgvs = ''
    
    for num,s,e,s_c,e_c in zip(ind_df.index, ind_df['start'], ind_df['end'], ind_df['start_coding_num'], 
                               ind_df['end_coding_num']):
        # for insertion (new pos (start/end of new exon) in intron)
        if new_pos == 0:
            if pos_last == e:
                e = pos_next-1
                new_pos +=1
                ref_n = ref_seq.fetch(chrom, pos_last-1, pos_last)
                alt_n = ref_seq.fetch(chrom, pos_last-1, pos_next-1)
                hgsv_pos = pos_last
                # include coding position and HGVS
                if ori == '+' or ori == 'plus':
                    continue
                elif ori == '-' or ori == 'minus':
                    cod_start_n = s_c - 1
                    cod_end_n = s_c
                    cod_diff = pos_next - pos_last - 1
                    #cdna_hgvs = f'{cod_start_n}_{cod_end_n}ins[{cod_start_n}-{cod_diff}_{cod_start_n}-1]'
                    cdna_hgvs = f'{cod_start_n}_{cod_end_n}ins[{cod_end_n}-{cod_diff}_{cod_end_n}-1]'
                    
            elif pos_next == s:
                s = pos_last+1
                new_pos +=1
                ref_n = ref_seq.fetch(chrom, pos_next-2, pos_next-1)
                alt_n = ref_n + ref_seq.fetch(chrom, pos_last, pos_next-1)
                hgsv_pos = pos_next-1
                # include coding position and HGVS
                if ori == '+' or ori == 'plus':
                    continue
                elif ori == '-' or ori == 'minus':
                    cod_start_n = e_c
                    cod_end_n = e_c + 1
                    cod_diff = pos_next - pos_last - 1
                    cdna_hgvs = f'{cod_start_n}_{cod_end_n}ins[{cod_start_n}+1_{cod_start_n}+{cod_diff}]'
                
            # for deletion (new pos (start/end of new exon) in exon)
            elif pos_next-1 == e:
                e = pos_last
                new_pos +=1
                ref_n = ref_seq.fetch(chrom, pos_last-1, pos_next-1)
                alt_n = ref_seq.fetch(chrom, pos_last-1, pos_last)
                hgsv_pos = pos_last
                # include coding position and HGVS
                if ori == '+' or ori == 'plus':
                    continue
                elif ori == '-' or ori == 'minus':
                    cod_diff = pos_next - pos_last - 1
                    cod_start_n = s_c
                    cod_end_n = s_c + cod_diff - 1
                    cdna_hgvs = f'{cod_start_n}_{cod_end_n}del'
                
            elif pos_last+1 == s:
                s = pos_next
                new_pos +=1
                ref_n = ref_seq.fetch(chrom, pos_last-1, pos_next-1)
                alt_n = ref_seq.fetch(chrom, pos_last-1, pos_last)
                hgsv_pos = pos_last
                # include coding position and HGVS
                if ori == '+' or ori == 'plus':
                    continue
                elif ori == '-' or ori == 'minus':
                    cod_diff = pos_next - pos_last - 1
                    cod_start_n = e_c - cod_diff + 1
                    cod_end_n = e_c
                    cdna_hgvs = f'{cod_start_n}_{cod_end_n}del'

        else:
            pass
        
        # e.g. in exon skipping events
        if s+1 > e:
            cdna += ''
        
        # if original variant is in new cDNA
        elif s <= var_pos and e >= var_pos:
            # for deletions starting in exon
            if ref_len>0:
                if ori == '+':
                    if var_pos+ref_len < e:
                        cdna += ref_seq.fetch(chrom, s-1, var_pos)
                        cdna += ref_seq.fetch(chrom, var_pos+ref_len, e)
                    else: # var_pos+ref_len >= e
                        cdna += ref_seq.fetch(chrom, s-1, var_pos)
                elif ori == '-' or ori == 'minus':
                    if var_pos+ref_len < e:
                        cdna += rev_comp(ref_seq.fetch(chrom, var_pos+ref_len, e))
                        cdna += rev_comp(ref_seq.fetch(chrom, s-1, var_pos))
                    else: # var_pos+ref_len >= e
                        cdna += rev_comp(ref_seq.fetch(chrom, s-1, var_pos))
                else:
                    pass

            # for insertions, substitutions
            else:
                if ori == '+':
                    cdna += ref_seq.fetch(chrom, s-1, var_pos-1)
                    cdna += variants.iloc[ind]['ALT']
                    cdna += ref_seq.fetch(chrom, var_pos, e)
                elif ori == '-' or ori == 'minus':
                    cdna += rev_comp(ref_seq.fetch(chrom, var_pos, e))
                    cdna += rev_comp(variants.iloc[ind]['ALT'])
                    cdna += rev_comp(ref_seq.fetch(chrom, s-1, var_pos-1))
                else:
                    pass
    
        # for deletions starting in intron
        elif ref_len>0 and s <= var_pos+ref_len and e >= var_pos+ref_len:
            if ori == '+':
                cdna += ref_seq.fetch(chrom, var_pos+ref_len, e)
            elif ori == '-' or ori == 'minus':
                cdna += rev_comp(ref_seq.fetch(chrom, var_pos+ref_len, e))
            else:
                pass
        
        
        
        
        elif ori == '+':
            cdna += ref_seq.fetch(chrom, s-1, e)
        elif ori == '-' or ori == 'minus':
            cdna += rev_comp(ref_seq.fetch(chrom, s-1, e))
        else:
            pass
    return [cdna, hgsv_pos, ref_n, alt_n, cdna_hgvs]

In [ ]:
# for reference cDNA/protein sequence for comparison
def ref_cdna(ind):
    cdna = ''
    chrom = str(variants.iloc[ind]['#CHROM'])
    ind_df = sdhb_gtf
    ori = list(ind_df['strand'])[0]
    for s,e in zip(ind_df['start'], ind_df['end']):
        if ori == '+':
            cdna += ref_seq.fetch(chrom, s-1, e)
        elif ori == '-' or ori == 'minus':
            cdna += rev_comp(ref_seq.fetch(chrom, s-1, e))
        else:
            pass
    return cdna

# position of change in nucleic sequence
def nuc_pos(ind):
    alt_nuc = variants.iloc[ind]['cDNA']
    ref_nuc = variants.iloc[ind]['ref_cDNA']
    len_indel = abs(len(alt_nuc)-len(ref_nuc))
    ct = 0
    nuc_pos = 0
    for a,r in zip(alt_nuc, ref_nuc[:len(alt_nuc)]):
        ct+=1
        if a != r:
            nuc_pos = ct
            break
        else:
            continue
    if nuc_pos == 0 and len(alt_nuc)<len(ref_nuc):
        nuc_pos = len(alt_nuc)+1
    elif nuc_pos == 0 and len(alt_nuc)==len(ref_nuc):
        nuc_pos = 'no_change'
    return [nuc_pos, len_indel]

# position of change in aa sequence
def prot_pos(ind):
    alt_prot = variants.iloc[ind]['termin_prot']
    ref_prot = variants.iloc[ind]['ref_protein_seq']
    ct = 0
    aa_pos = 0
    for a,r in zip(alt_prot, ref_prot[:len(alt_prot)]):
        ct+=1
        if a != r:
            aa_pos = ct
            break
        else:
            continue
    if aa_pos == 0 and len(alt_prot)<len(ref_prot):
        aa_pos = len(alt_prot)+1
    elif aa_pos == 0 and len(alt_prot)==len(ref_prot):
        aa_pos = 'no_change'
    return aa_pos

# length of exons
def exon_len(ind):
    ind_df = sdhb_gtf
    exon_lst = []
    bott = 0
    for s,e,n in zip(ind_df['start'], ind_df['end'], ind_df['exon_number']):
        exon_lst.append((n, bott+1, bott+abs(s-e)+1))
        bott += abs(s-e)+1
    return exon_lst, n

# in which exon change occurs
def changed_exon(ind):
    exon = 0
    nuc_change_pos = variants.iloc[ind]['nuc_change_pos']
    if type(nuc_change_pos) != 'no_change':
        for ex,s,e in variants.iloc[ind]['exons_lengths']:
            if nuc_change_pos >= s and nuc_change_pos <= e:
                exon = ex
            else:
                pass
    return exon

In [ ]:
# find out next matching amino acids position
import re
def aas_next(ind):
    change_pos = variants.iloc[ind]['prot_change_pos']
    ref_n = variants.iloc[ind]['ref_n']
    alt_n = variants.iloc[ind]['alt_n']
    len_indel_nuc = abs(len(ref_n)-len(alt_n))
    rest_3 = len_indel_nuc%3
    ct = 0
    pos_next_normal = ''
    len_indel = 0
    
    try:
        if type(change_pos)!='no_change':
            alt1 = variants.iloc[ind]['termin_prot']
            short_alt = alt1[change_pos-1:]
            ref1 = variants.iloc[ind]['ref_protein_seq']
            short_ref = ref1[change_pos-1:]
            kind = variants.iloc[ind]['kind']
            
            if kind=='ins':
                if rest_3 == 0:
                    if re.search(short_ref, short_alt) and len(alt1)>len(ref1):
                        len_indel = re.search(short_ref, short_alt).regs[0][0]
                        pos_next_normal = change_pos+len_indel
                    elif re.search(short_ref[1:], short_alt) and len(alt1)>len(ref1):
                        len_indel = re.search(short_ref[1:], short_alt).regs[0][0]
                        pos_next_normal = change_pos+len_indel
                else:
                    pos_next_normal = 'probably frameshift'
            
            elif kind =='del':
                if rest_3 == 0:
                    if re.search(short_alt, short_ref) and len(ref1)>len(alt1):
                        len_indel = re.search(short_alt, short_ref).regs[0][0]
                        pos_next_normal = change_pos+len_indel
                    elif re.search(short_alt[1:], short_ref) and len(ref1)>len(alt1):
                        len_indel = re.search(short_alt[1:], short_ref).regs[0][0]
                        pos_next_normal = change_pos+len_indel
                else:
                    pos_next_normal = 'probably frameshift'
            else:
                pass
        else:
            pos_next_normal = 'no change'
        if abs(len(alt1)-len(ref1)) > (len_indel+1):
            pos_next_normal = 'probably frameshift'
            
        else:
            pass
    except:
        pos_next_normal = 'error'
    return [pos_next_normal, len_indel]

In [ ]:
# new columns in variants df with cDNA, protein sequence
variants[['cDNA', 'hgsv_pos', 'ref_n', 'alt_n', 'cDNA_HGVS']] = [cDNA(i) for i in variants.index]
variants['protein_seq'] = [str(Seq(cDNA(i)[0]).translate()) for i in variants.index]
# terminated protein sequence after first stop codon
variants['termin_prot'] = variants['protein_seq'].str.split('*', expand=True)[0]

variants['cDNA_HGVS'] = 'NM_003000:r.' + variants['cDNA_HGVS']

# new columns in variants df with reference cDNA, protein sequence
variants['ref_cDNA'] = [ref_cdna(i) for i in variants.index]
variants['ref_protein_seq'] = [str(Seq(ref_cdna(i)).translate()) for i in variants.index]

# length of sequences
for i in ['cDNA', 'termin_prot', 'ref_cDNA', 'ref_protein_seq']:
    variants[i+'_len'] = variants[i].str.len()

# position of change in nucleic/aa sequence
variants[['nuc_change_pos', 'len_indel_nuc']] = [nuc_pos(i) for i in variants.index]
variants['prot_change_pos'] = [prot_pos(i) for i in variants.index]

# position of next matching aa and length of indel
variants[['aa_next_pos', 'len_indel_substit_prot']] = [aas_next(i) for i in variants.index]

# length of exons
variants[['exons_lengths', 'max_exon']] = [exon_len(i) for i in variants.index]

# changed exon
variants['changed_exon'] = [changed_exon(i) for i in variants.index]

In [ ]:
def HGVS_protein(ind):
    one_to_three = {
    'A': 'Ala', 'R': 'Arg', 'N': 'Asn', 'D': 'Asp', 'C': 'Cys',
    'E': 'Glu', 'Q': 'Gln', 'G': 'Gly', 'H': 'His', 'I': 'Ile',
    'L': 'Leu', 'K': 'Lys', 'M': 'Met', 'F': 'Phe', 'P': 'Pro',
    'S': 'Ser', 'T': 'Thr', 'W': 'Trp', 'Y': 'Tyr', 'V': 'Val',
    '*': 'Ter'}
    ref1 = variants.iloc[ind]['ref_protein_seq']
    alt1 = variants.iloc[ind]['protein_seq']
    termin_prot = variants.iloc[ind]['termin_prot']
    prot_change_pos = variants.iloc[ind]['prot_change_pos']
    aa_next_pos = variants.iloc[ind]['aa_next_pos']
    len_indel_prot = variants.iloc[ind]['len_indel_substit_prot']
    ins_del_type = variants.iloc[ind]['kind']
    
    if prot_change_pos != 'no_change':
        # len_indel = 0 -> "probably frameshift"
        if len_indel_prot == 0:
            ref_aa = one_to_three[ref1[prot_change_pos-1]]
            alt_aa = one_to_three[alt1[prot_change_pos-1]]
            if alt_aa == 'Ter':
                prot_hgvs = f'NP_002991.2:p.{ref_aa}{prot_change_pos}{alt_aa}'
            else:
                prot_hgvs = f'NP_002991.2:p.{ref_aa}{prot_change_pos}{alt_aa}'
                if aa_next_pos == 'probably frameshift':
                    prot_hgvs += 'fs'
                    if len(termin_prot) < len(ref1):
                        len_to_termin = len(termin_prot) - prot_change_pos +1
                        prot_hgvs += f'Ter{len_to_termin}'
        # insertion or deletion with less/equal 3 aas inserted/deleted
        elif len_indel_prot <= 3:
            # no termination codon in inserted sequence
            if ins_del_type == 'ins' and '*' not in alt1:
                ref_aa_start = one_to_three[ref1[prot_change_pos-1]]
                ref_aa_end = one_to_three[ref1[prot_change_pos]]
                ins_aas = ''
                for i in range(len_indel_prot):
                    ins_aas += one_to_three[alt1[prot_change_pos-1+i]]
                prot_hgvs = f'NP_002991.2:p.{ref_aa_start}{prot_change_pos}_{ref_aa_end}{prot_change_pos+1}ins{ins_aas}'
            # termination codon in inserted sequence
            elif ins_del_type == 'ins' and '*' in alt1:
                ref_aa_start = one_to_three[ref1[prot_change_pos-1]]
                ref_aa_end = one_to_three[ref1[prot_change_pos]]
                ins_to_termin_len = len(termin_prot) - prot_change_pos
                ins_aas = ''
                if ins_to_termin_len == 0:
                     prot_hgvs = f'NP_002991.2:p.{ref_aa_start}{prot_change_pos}_{ref_aa_end}{prot_change_pos+1}insTer'
                else:
                    for i in range(ins_to_termin_len):
                        ins_aas += one_to_three[alt1[prot_change_pos-1+i]]
                    prot_hgvs = f'NP_002991.2:p.{ref_aa_start}{prot_change_pos}_{ref_aa_end}{prot_change_pos+1}ins{ins_aas}'
            # deleted aas
            elif ins_del_type == 'del':
                if len_indel_prot == 1:
                    del_aa = one_to_three[ref1[prot_change_pos-1]]
                    prot_hgvs = f'NP_002991.2:p.{del_aa}{prot_change_pos}del'
                else:
                    del_aa1 = one_to_three[ref1[prot_change_pos-1]]
                    del_aa2 = one_to_three[ref1[prot_change_pos-1 + len_indel_prot-1]]
                    prot_hgvs = f'NP_002991.2:p.{del_aa1}{prot_change_pos}_{del_aa2}{prot_change_pos+len_indel_prot-1}del'
        # insertion or deletion with more than 3 aas inserted/deleted
        else:
            # no termination codon in inserted sequence
            if ins_del_type == 'ins' and '*' not in alt1:
                ref_aa_start = one_to_three[ref1[prot_change_pos-1]]
                ref_aa_end = one_to_three[ref1[prot_change_pos]]
                prot_hgvs = f'NP_002991.2:p.{ref_aa_start}{prot_change_pos}_{ref_aa_end}{prot_change_pos+1}insX[{len_indel_prot}]'
            # termination codon in inserted sequence
            elif ins_del_type == 'ins' and '*' in alt1:
                ref_aa_start = one_to_three[ref1[prot_change_pos-1]]
                ref_aa_end = one_to_three[ref1[prot_change_pos]]
                ins_to_termin_len = len(termin_prot) - prot_change_pos
                ins_aas = ''
                if ins_so_termin_len == 0:
                    prot_hgvs = f'NP_002991.2:p.{ref_aa_start}{prot_change_pos}_{ref_aa_end}{prot_change_pos+1}insTer'
                elif ins_so_termin_len <= 3:
                    for i in range(ins_to_termin_len):
                        ins_aas += one_to_three[alt1[prot_change_pos-1+i]]
                    prot_hgvs = f'NP_002991.2:p.{ref_aa_start}{prot_change_pos}_{ref_aa_end}{prot_change_pos+1}ins{ins_aas}'
                else:
                    prot_hgvs = f'NP_002991.2:p.{ref_aa_start}{prot_change_pos}_{ref_aa_end}{prot_change_pos+1}insTer{ins_to_termin_len}'
                
            # deleted aas
            elif ins_del_type == 'del':
                del_aa1 = one_to_three[ref1[prot_change_pos-1]]
                del_aa2 = one_to_three[ref1[prot_change_pos-1 + len_indel_prot-1]]
                prot_hgvs = f'NP_002991.2:p.{del_aa1}{prot_change_pos}_{del_aa2}{prot_change_pos+len_indel_prot-1}del'
    else:
        prot_hgvs = 'no_change'
    return prot_hgvs
    

In [ ]:
# protein HGVS
variants['protein_HGVS'] = [HGVS_protein(i) for i in variants.index]

In [ ]:
variants.to_pickle('01_dataframes/'+prefix_variant_file+'_cDNA_protSeq')

In [ ]:
variants.to_excel('02_output_analysis/01_cDNA_table/'+prefix_variant_file+'_cDNA_protSeq.xlsx', index=False)